## GDPR in practice: data anonymization

### Some dependencies and some data

In [ ]:
import os
from random import sample
from urllib.request import urlopen
import numpy as np
import itertools
from tqdm import tqdm, trange
import sklearn as sk
import sklearn.linear_model
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset

import logging
logger = logging.getLogger(__name__)

In [ ]:
url = "https://raw.githubusercontent.com/glample/tagger/master/dataset/eng.testa"
html = urlopen(url)
with open('eng.testa', 'a') as the_file:
    for line in html:
        the_file.write(line.decode('UTF-8'))

### Some useful functions

In [ ]:
class InputExample(object):
    """A single training/test example for token classification."""

    def __init__(self, guid, words, labels):
        """Constructs a InputExample.
        Args:
            guid: Unique id for the example.
            words: list. The words of the sequence.
            labels: (Optional) list. The labels for each word of the sequence. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.words = words
        self.labels = labels


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_ids):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_ids = label_ids


def read_examples_from_file(data_dir, mode="eng.testa"):
    """Creating InputExamples out of a file"""
    file_path = os.path.join(data_dir, "{}".format(mode))
    guid_index = 1
    examples = []
    with open(file_path, encoding="utf-8") as f:
        words = []
        labels = []
        for line in f:
            if line.startswith("-DOCSTART-") or line == "" or line == "\n":
                if words:
                    examples.append(InputExample(guid="{}-{}".format(mode, guid_index), words=words, labels=labels))
                    guid_index += 1
                    words = []
                    labels = []
            else:
                splits = line.split(" ")
                words.append(splits[0])
                if len(splits) > 1:
                    labels.append(splits[-1].replace("\n", ""))
                else:
                    # Examples could have no label for mode = "test"
                    labels.append("O")
        if words:
            examples.append(InputExample(guid="{}-{}".format(mode, guid_index), words=words, labels=labels))
    return examples


def convert_examples_to_features(
    examples,
    label_list,
    max_seq_length,
    tokenizer,
    cls_token_at_end=False,
    cls_token="[CLS]",
    cls_token_segment_id=1,
    sep_token="[SEP]",
    sep_token_extra=False,
    pad_on_left=False,
    pad_token=0,
    pad_token_segment_id=0,
    pad_token_label_id=-100,
    sequence_a_segment_id=0,
    mask_padding_with_zero=True,
):
    """ Loads a data file into a list of `InputBatch`s
        `cls_token_at_end` define the location of the CLS token:
            - False (Default, BERT/XLM pattern): [CLS] + A + [SEP] + B + [SEP]
            - True (XLNet/GPT pattern): A + [SEP] + B + [SEP] + [CLS]
        `cls_token_segment_id` define the segment id associated to the CLS token (0 for BERT, 2 for XLNet)
    """

    label_map = {label: i for i, label in enumerate(label_list)}

    features = []
    for (ex_index, example) in enumerate(examples):
        if ex_index % 10000 == 0:
            logger.info("Writing example %d of %d", ex_index, len(examples))

        tokens = []
        label_ids = []
        for word, label in zip(example.words, example.labels):
            word_tokens = tokenizer.tokenize(word)
            tokens.extend(word_tokens)
            # Use the real label id for the first token of the word, and padding ids for the remaining tokens
            label_ids.extend([label_map[label]] + [pad_token_label_id] * (len(word_tokens) - 1))

        # Account for [CLS] and [SEP] with "- 2" and with "- 3" for RoBERTa.
        special_tokens_count = 3 if sep_token_extra else 2
        if len(tokens) > max_seq_length - special_tokens_count:
            tokens = tokens[: (max_seq_length - special_tokens_count)]
            label_ids = label_ids[: (max_seq_length - special_tokens_count)]

        # The convention in BERT is:
        # (a) For sequence pairs:
        #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
        #  type_ids:   0   0  0    0    0     0       0   0   1  1  1  1   1   1
        # (b) For single sequences:
        #  tokens:   [CLS] the dog is hairy . [SEP]
        #  type_ids:   0   0   0   0  0     0   0
        #
        # Where "type_ids" are used to indicate whether this is the first
        # sequence or the second sequence. The embedding vectors for `type=0` and
        # `type=1` were learned during pre-training and are added to the wordpiece
        # embedding vector (and position vector). This is not *strictly* necessary
        # since the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.
        #
        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        tokens += [sep_token]
        label_ids += [pad_token_label_id]
        if sep_token_extra:
            # roberta uses an extra separator b/w pairs of sentences
            tokens += [sep_token]
            label_ids += [pad_token_label_id]
        segment_ids = [sequence_a_segment_id] * len(tokens)

        if cls_token_at_end:
            tokens += [cls_token]
            label_ids += [pad_token_label_id]
            segment_ids += [cls_token_segment_id]
        else:
            tokens = [cls_token] + tokens
            label_ids = [pad_token_label_id] + label_ids
            segment_ids = [cls_token_segment_id] + segment_ids

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding_length = max_seq_length - len(input_ids)
        if pad_on_left:
            input_ids = ([pad_token] * padding_length) + input_ids
            input_mask = ([0 if mask_padding_with_zero else 1] * padding_length) + input_mask
            segment_ids = ([pad_token_segment_id] * padding_length) + segment_ids
            label_ids = ([pad_token_label_id] * padding_length) + label_ids
        else:
            input_ids += [pad_token] * padding_length
            input_mask += [0 if mask_padding_with_zero else 1] * padding_length
            segment_ids += [pad_token_segment_id] * padding_length
            label_ids += [pad_token_label_id] * padding_length

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length
        assert len(label_ids) == max_seq_length

        if ex_index < 5:
            logger.info("*** Example ***")
            logger.info("guid: %s", example.guid)
            logger.info("tokens: %s", " ".join([str(x) for x in tokens]))
            logger.info("input_ids: %s", " ".join([str(x) for x in input_ids]))
            logger.info("input_mask: %s", " ".join([str(x) for x in input_mask]))
            logger.info("segment_ids: %s", " ".join([str(x) for x in segment_ids]))
            logger.info("label_ids: %s", " ".join([str(x) for x in label_ids]))

        features.append(
            InputFeatures(input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids, label_ids=label_ids)
        )
    return features

### A direct, pre-trained NER approach

In [ ]:
from transformers import pipeline

In [ ]:
nlp = transformers.pipeline("ner")

In [ ]:
nlp("Adrien Ehrhardt works on data-anonymization with Abderrahim MAMA")

We'd like to detect "Adrien Ehrhardt" and anonymize this part, hence detect the tokens == input examples 'Ad', '##rien', 'E', '##hr', '##hardt' as being of class PER.

### Obtaining the representation of the dataset from Bert

#### An example

In [ ]:
from transformers import BertTokenizer, BertModel
import torch

In [ ]:
model = BertModel.from_pretrained("bert-large-cased")
tokenizer = BertTokenizer.from_pretrained("bert-large-cased")

sequence = "Adrien Ehrhardt donne un projet d'informatique aux étudiants de INF442."
tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sequence)))
inputs = tokenizer.encode(sequence, return_tensors="pt")

outputs = model(inputs)[0]

In [ ]:
outputs.detach().numpy()[0].shape

In [ ]:
outputs

Each line is the 1024-dimensional representation of each of the 28 tokens in the sentence (out of a vocabulary size of 30k+).

#### The real thing: doing the same for the CoNLL03 dataset

In [ ]:
pad_token_label_id = CrossEntropyLoss().ignore_index
# The labels in CoNLL03
##labels = ["O", "B-MISC", "I-MISC", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC"]
# The labels in 

# The model we'll use
model = BertModel.from_pretrained("bert-large-cased")
tokenizer = BertTokenizer.from_pretrained("bert-large-cased")
# Reading the file
examples = read_examples_from_file(".", mode="Data/Data/eng.train")

In [ ]:
# Converting the words in tokens
features = convert_examples_to_features(
            examples,
            label_list=labels,
            max_seq_length=128,
            tokenizer=tokenizer,
            cls_token_at_end=False,
            cls_token=tokenizer.cls_token,
            cls_token_segment_id=0,
            sep_token=tokenizer.sep_token,
            sep_token_extra=False,
            pad_on_left=False,
            pad_token=tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
            pad_token_segment_id=0,
            pad_token_label_id=pad_token_label_id)

In [ ]:
# Convert to Tensors and build dataset
all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
all_label_ids = torch.tensor([f.label_ids for f in features], dtype=torch.long)

dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)

In [ ]:
# As this is too computationally intensive, we'll do this by batch of 64 tokens
eval_sampler = SequentialSampler(dataset)
eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=64)

In [ ]:
preds = None
# This tells the model to only "evaluate" (forward-pass)
model.eval()
for batch in tqdm(eval_dataloader, desc="Evaluating"):
    batch = tuple(t.to("cpu") for t in batch)
    with torch.no_grad():  # do not calculate gradients, as we won't be doing backward propagation
        inputs = {"input_ids": batch[0], 
                  "attention_mask": batch[1],
                  "token_type_ids": batch[2]}
        outputs = model(**inputs)
        last_hidden_layer = outputs[0]  # the representation we're interested in is the output of the last hidden layer
    if preds is None:
        preds = last_hidden_layer.detach().cpu().numpy()
        out_label_ids = batch[3].detach().cpu().numpy()
    else:
        preds = np.append(preds, last_hidden_layer.detach().cpu().numpy(), axis=0)
        out_label_ids = np.append(out_label_ids, batch[3], axis=0)

In [ ]:
out_label_list = [[] for _ in range(out_label_ids.shape[0])]
preds_list = [[] for _ in range(out_label_ids.shape[0])]

label_map = {i: label for i, label in enumerate(labels)}
        
for i in range(out_label_ids.shape[0]):
    for j in range(out_label_ids.shape[1]):
        if out_label_ids[i, j] != pad_token_label_id:
            out_label_list[i].append(label_map[out_label_ids[i][j]])
            preds_list[i].append(preds[i][j])

`out_label_list` is of shape (#number of sentences, #tokens in this sentence, 1)

`preds_list` is of shape (#number of sentences, #tokens in this sentence, 1024 hidden representations)

For the sake of simplicity, we'll consider that an observation / a sample is a token, not a sentence (the contextual meaning of each token is already taken into account in its representation), so we need to "flatten" both lists so that they're of shape (#tokens, 1)

In [ ]:
true_labels = np.array(list(itertools.chain.from_iterable(out_label_list)))

In [ ]:
flat_list = []
for sentence in preds_list:
    for token in sentence:
        flat_list.append(token)
        
representation = np.array(flat_list)

In [ ]:
true_labels.shape

In [ ]:
representation.shape

In [ ]:
np.save("../Data/true_labels.train.npy", true_labels)

In [ ]:
np.save("../Data/representation.train.npy", representation)

### Example of a simple classifier on top

Now that we've covered the hard part (converting words to a useful numerical representation), we can classify them as we would for iris flowers (e.g. with a Logistic Regression - here with cross-validation)!

In [ ]:
model_top = sk.linear_model.LogisticRegressionCV()

In [ ]:
model_top.fit(X=representation, y=true_labels)

In [ ]:
model_top.score(X=representation, y=true_labels)

### Loading the data: example

If you plan to use some Python in your project, here's how you can load the numpy data:

In [ ]:
true_labels = np.load("../Data/true_labels.train.npy")

In [ ]:
true_labels

In [ ]:
representation = np.load("../Data/representation.train.npy")

In [ ]:
representation

### Loading and converting to CSV: files get too big, you'll have to do this yourself and sample!

In [ ]:
representation = np.load("representation.train.npy")
# SAMPLE
int_train = sample(range(representation.shape[0]), 10000)
np.savetxt("representation.train.csv", representation[int_train,:], delimiter=",")
true_labels = np.load("true_labels.train.npy")
np.savetxt("true_labels.train.csv", true_labels[int_train], delimiter=",", fmt='%s')

In [ ]:
representation = np.load("representation.testa.npy")
# SAMPLE
int_train = sample(range(representation.shape[0]), 2000)
np.savetxt("representation.testa.csv", representation[int_train,:], delimiter=",")
true_labels = np.load("true_labels.testa.npy")
np.savetxt("true_labels.testa.csv", true_labels[int_train], delimiter=",", fmt='%s')

In [ ]:
representation = np.load("representation.testb.npy")
# SAMPLE
int_train = sample(range(representation.shape[0]), 2000)
np.savetxt("representation.testb.csv", representation[int_train,:], delimiter=",")
true_labels = np.load("true_labels.testb.npy")
np.savetxt("true_labels.testb.csv", true_labels[int_train], delimiter=",", fmt='%s')